In [7]:
import numpy as np
import re
import pandas as pd

In [8]:
#Acquisition swatchs
kml_file = "Data/In/Sentinel-2A_MP_ACQ_KML_20200326T120000_20200413T150000.kml"

In [9]:
#Converts the kml file into one txt string with urf-8 encoding
kml_str = ""
for line in open(kml_file):
    kml_str += line

In [10]:
#Get rid of the blanks
kml_str = kml_str.replace("\n", "")
kml_str = kml_str.replace("\t","")

In [11]:
#All coordinates and timestamps are stored in between the markers below (designated by .*?)
coords_pattern = '<coordinates>(.*?)</coordinates>'
timespan_pattern = '<TimeSpan>(.*?)</TimeSpan>'

#Extracts the coordinates and times between their markers
coords = np.array(re.findall(coords_pattern, kml_str, re.DOTALL))
times = np.array(re.findall(timespan_pattern, kml_str, re.DOTALL))

In [12]:
overpasses = []

#Extracts just the start date, stop date and times of day
for row in times:
    start = row.find("<begin>") + 7
    stop = row.find("T")
    overpasses.append(row[start:stop])

In [65]:
#Reformat the coordinates: 
##"coordX1,coordY1/coordX2,coordY2" instead of xml style "coordX1,coordY1,0 coordX2,coordY2"

for row_index in range(coords.shape[0]):
    coords[row_index] = coords[row_index].replace(",0 ", "/")

In [106]:
polygon = []
polygons = []
coord =''

#Creates 2d array of all polygons with each cell being a polygon
for row in coords:
    for c in row:   
        if c!='/':
            coord = coord+c
        else: 
            polygon.append(coord)
            coord = ''
            
    polygons.append(polygon)
    polygon = []

    

In [113]:
#Final data frame
df = pd.DataFrame({"Polygon": polygons, "Overpass": overpasses}, index=range(len(polygons)))

In [116]:
df.to_csv("parsedkml.csv")